In [1]:
import pandas as pd

In [23]:
def to_int(df):
    # strip all ',' from string numbers
    df = df.apply(lambda x: x.replace(',', '') if type(x) == str else x)
    return df.astype(float)

def get_codes(geo):
    geo = geo.split('US')[1]
    state = geo[:2]
    county = geo[2:5]
    tract = geo[5:len(geo) - 1]
    block = geo[-1]
    return state, county, tract, block

def get_county_name(geo_area_name):
    county_name = geo_area_name.split(';')[2].strip()
    return county_name
def get_state_name(geo_area_name):
    state_name = geo_area_name.split(';')[3].strip()
    return state_name
    

In [27]:
df = pd.read_csv("/Users/jack/Downloads/B23025 Employment Status 2024/ACSDT5Y2022.B23025-Data.csv", skiprows=1)
cols = ["Geography", "Geographic Area Name"] + [col for col in df.columns if "Estimate" in col]
rename_mapper = {
    "Estimate!!Total:": "Total",
    "Estimate!!Total:!!In labor force:": "In labor force",
    "Estimate!!Total:!!In labor force:!!Civilian labor force:": "Civilian labor force",
    "Estimate!!Total:!!In labor force:!!Civilian labor force:!!Employed": "Employed",
    "Estimate!!Total:!!In labor force:!!Civilian labor force:!!Unemployed": "Unemployed",
    "Estimate!!Total:!!In labor force:!!Armed Forces": "Armed Forces",
    "Estimate!!Total:!!Not in labor force": "Not in labor force"
}
df = df[cols].rename(columns=rename_mapper)
df['State'], df['County'], df['Tract'], df['Block'] = zip(*df['Geography'].map(get_codes))
df['County Name'] = df['Geographic Area Name'].map(get_county_name)
df['State Name'] = df['Geographic Area Name'].map(get_state_name)
df = df.drop(columns=['Geography', 'Geographic Area Name'])
df['Unemployment Rate'] = df['Unemployed'] / df['In labor force']
df.to_csv("bg_unemployment", index=False)

## Income

In [33]:
df = pd.read_csv("/Users/jack/Downloads/Median Household Income 2024/ACSDT5Y2022.B19013-Data.csv", skiprows=1)
df = df[['Geography', 'Geographic Area Name', 'Estimate!!Median household income in the past 12 months (in 2022 inflation-adjusted dollars)']].rename(columns={'Estimate!!Median household income in the past 12 months (in 2022 inflation-adjusted dollars)': 'Median Household Income'})
df['State'], df['County'], df['Tract'], df['Block'] = zip(*df['Geography'].map(get_codes))
df['County Name'] = df['Geographic Area Name'].map(get_county_name)
df['State Name'] = df['Geographic Area Name'].map(get_state_name)
df = df.drop(columns=['Geography', 'Geographic Area Name'])
df.to_csv("bg_income", index=False)